# Model Save and Load

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(7777)
tf.random.set_seed(7777)

In [3]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])
        
        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(16, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)
    
    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)
    
    net1_1 = Conv2D(32, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])
    
    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)
    
    net = Add()([net, net3])
    
    net = MaxPool2D()(net)
    
    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')
    
    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 16)   448         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 8, 8, 16)     0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 32)     544         ['max_pooling2d[0][0]']          
                                                                                             

In [5]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

### save 함수

In [6]:
model.save('checkpoints/model.h5')
                                # .h5 파일 확장자 : tf에서는 널리 사용되는 확장자

In [8]:
model = tf.keras.models.load_model('checkpoints/model.h5')

In [9]:
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 16)   448         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 8, 8, 16)     0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 32)     544         ['max_pooling2d[0][0]']          
                                                                                             

### save_weights 함수 

 - weights만 저장 하므로, 저장공간이 절약됨. 

In [11]:
model.save_weights('checkpoints/model_weight.h5')

In [13]:
# tf.keras.models.load_model('checkpoints/model_weight.h5')
model = build_resnet((32, 32, 3))

In [14]:
model.load_weights('checkpoints/model_weight.h5')

In [15]:
model

### Callbacks 함수 사용하기

In [16]:
save_path = 'checkpoints/{epoch} - {val_loss: .2f}.h5'

In [17]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, \
    monitor='val_accuracy', save_best_only=True)
                            # 최고 성능만 저장한다.(True)

In [18]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer = opt, loss=loss, metrics=['accuracy'])

In [19]:
model.fit(
    train_x,
    train_y,
    epochs = 1,
    validation_data = (test_x, test_y),
    callbacks = [checkpoint]
)

1563/1563 [==============================] - 28s 17ms/step - loss: 2.3375 - accuracy: 0.1002 - val_loss: 2.3243 - val_accuracy: 0.1000


### pb 형식으로 저장 하기 
 - 모델을 protoBuffer 형식으로 저장

In [ ]:
# pb는 weight와 모델 구조를 같이 저장할때 쓴다.

In [27]:
save_path = 'checkpoints/{epoch}-{val_loss:.2f}'

In [28]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, \
    monitor='val_accuracy', save_best_only=True)
                            # 최고 성능만 저장한다.(True)

learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.compile(optimizer = opt, loss=loss, metrics=['accuracy'])


model.fit(
    train_x,
    train_y,
    epochs = 1,
    validation_data = (test_x, test_y),
    callbacks = [checkpoint]
)

1563/1563 [==============================] - 23s 15ms/step - loss: 2.3418 - accuracy: 0.0993 - val_loss: 2.3844 - val_accuracy: 0.1000


In [32]:
!ls checkpoints/1-2.38

assets            keras_metadata.pb saved_model.pb    variables


In [34]:
model = tf.saved_model.load('checkpoints/1-2.38')

In [35]:
model(test_x)

<tf.Tensor: shape=(10000, 10), dtype=float32, numpy=
array([[0.09696754, 0.07097792, 0.14373048, ..., 0.0864548 , 0.05998822,
        0.11755131],
       [0.09696754, 0.07097792, 0.14373048, ..., 0.0864548 , 0.05998822,
        0.11755131],
       [0.09696754, 0.07097792, 0.14373048, ..., 0.0864548 , 0.05998822,
        0.11755131],
       ...,
       [0.09696754, 0.07097792, 0.14373048, ..., 0.0864548 , 0.05998822,
        0.11755131],
       [0.09696754, 0.07097792, 0.14373048, ..., 0.0864548 , 0.05998822,
        0.11755131],
       [0.09696754, 0.07097792, 0.14373048, ..., 0.0864548 , 0.05998822,
        0.11755131]], dtype=float32)>